<a href="https://colab.research.google.com/github/kevinwjin/pytorch-intro/blob/main/pytorchIntro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize, Compose
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
A = [[6, 9, 2],
     [3, 3, 7],
     [1, 0, 3]]
A_tensor = torch.tensor(A)
A

In [ ]:
B = np.array([0, 1, 2, 3])
B_tensor = torch.from_numpy(B)
B

In [ ]:
C = torch.zeros(4, 4)
C

In [ ]:
D = torch.ones(3, 3, 2, dtype = torch.int)
D

In [ ]:
A_tensor_zeros = torch.zeros_like(A_tensor)
A_tensor_zeros

In [ ]:
# the dtype argument allows you to explicitly specify the datatype of the tensor
A_tensor_rand = torch.rand_like(A_tensor, dtype = torch.float)
A_tensor_rand

In [ ]:
A_tensor_rand.dtype
A_tensor_rand.shape
A_tensor_rand.device

In [ ]:
from torchvision.datasets import FashionMNIST
from torchtext.datasets import AmazonReviewFull
from torchaudio.datasets import SPEECHCOMMANDS

training_data = FashionMNIST(
    # directory you want to store the dataset, can be a string e.g. "data"
    root = data_directory,
    # if False, will give you test set instead
    train = True,
    # download dataset if not already available in the root path you specified
    download = True,
    # transform images to tensor data structures to PyTorch can use them for training
    transform = ToTensor()
)

In [ ]:
# True if a GPU (instead of CPU) is available to train on
torch.cuda.is_available()